In [ ]:
import numpy as np
import torch
from torch import nn
import torch.nn.functional as F

In [ ]:
with open('/content/anna.txt', 'r') as f:
    text = f.read()

In [ ]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [ ]:
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

encoded = np.array([char2int[ch] for ch in text])

In [ ]:
encoded[:100]

array([79, 31, 45, 33, 63,  3,  8, 64, 30, 14, 14, 14, 11, 45, 33, 33, 27,
       64, 80, 45, 24, 56, 61, 56,  3, 55, 64, 45,  8,  3, 64, 45, 61, 61,
       64, 45, 61, 56, 74,  3, 51, 64,  3,  5,  3,  8, 27, 64, 65, 25, 31,
       45, 33, 33, 27, 64, 80, 45, 24, 56, 61, 27, 64, 56, 55, 64, 65, 25,
       31, 45, 33, 33, 27, 64, 56, 25, 64, 56, 63, 55, 64, 46,  2, 25, 14,
        2, 45, 27, 62, 14, 14, 72,  5,  3,  8, 27, 63, 31, 56, 25])

In [ ]:
def one_hot_encode(arr, n_labels):
    
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [ ]:
test_seq = np.array([[7, 2, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 0. 0. 0. 0. 1.]
  [0. 0. 1. 0. 0. 0. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [ ]:
def get_batches(arr, batch_size, seq_length):
    
    batch_size_total = batch_size * seq_length
    n_batches = len(arr)//batch_size_total
    
    arr =  arr[:n_batches * batch_size_total]
    
    arr = arr.reshape((batch_size, -1))
    
    for n in range(0, arr.shape[1], seq_length):
        x = arr[:, n:n+seq_length]
        
        y = np.zeros_like(x)
        yield x, y

In [ ]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [ ]:
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[79 31 45 33 63  3  8 64 30 14]
 [55 46 25 64 63 31 45 63 64 45]
 [ 3 25 42 64 46  8 64 45 64 80]
 [55 64 63 31  3 64 52 31 56  3]
 [64 55 45  2 64 31  3  8 64 63]
 [52 65 55 55 56 46 25 64 45 25]
 [64 78 25 25 45 64 31 45 42 64]
 [16 75 61 46 25 55 74 27 62 64]]

y
 [[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [ ]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        
        r_output, hidden = self.lstm(x, hidden)
        
        out = self.dropout(r_output)
        
        
        out = out.contiguous().view(-1, self.n_hidden)
        
        out = self.fc(out)
        
        return out, hidden
    
    
    def init_hidden(self, batch_size):
       
        weight = next(self.parameters()).data
        
        
        hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden

In [ ]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
   
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    
    
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
           

         
            h = tuple([each.data for each in h])

            net.zero_grad()
            
            output, h = net(inputs, h)
            
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            if counter % print_every == 0:
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                  
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train()
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

In [ ]:
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(83, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=83, bias=True)
)


In [ ]:


batch_size = 128
seq_length = 100
n_epochs = 20 

train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)



Epoch: 1/20... Step: 10... Loss: 0.0004... Val Loss: 0.0076
Epoch: 1/20... Step: 20... Loss: 0.0000... Val Loss: 0.0047
Epoch: 1/20... Step: 30... Loss: 0.0000... Val Loss: 0.0041
Epoch: 1/20... Step: 40... Loss: 0.0000... Val Loss: 0.0039
Epoch: 1/20... Step: 50... Loss: 0.0000... Val Loss: 0.0038
Epoch: 1/20... Step: 60... Loss: 0.0000... Val Loss: 0.0038
Epoch: 1/20... Step: 70... Loss: 0.0000... Val Loss: 0.0038
Epoch: 1/20... Step: 80... Loss: 0.0000... Val Loss: 0.0038
Epoch: 1/20... Step: 90... Loss: 0.0000... Val Loss: 0.0038
Epoch: 1/20... Step: 100... Loss: 0.0000... Val Loss: 0.0038
Epoch: 1/20... Step: 110... Loss: 0.0000... Val Loss: 0.0038
Epoch: 1/20... Step: 120... Loss: 0.0000... Val Loss: 0.0038
Epoch: 1/20... Step: 130... Loss: 0.0000... Val Loss: 0.0038
Epoch: 2/20... Step: 140... Loss: 0.0564... Val Loss: 0.0038
Epoch: 2/20... Step: 150... Loss: 0.0000... Val Loss: 0.0037
Epoch: 2/20... Step: 160... Loss: 0.0000... Val Loss: 0.0037
Epoch: 2/20... Step: 170... Loss:

In [ ]:
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [ ]:
def predict(net, char, h=None, top_k=None):
   
        
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
       
        h = tuple([each.data for each in h])
        out, h = net(inputs, h)

        p = F.softmax(out, dim=1).data
       
        
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        return net.int2char[char], h

In [ ]:
def sample(net, size, prime='The', top_k=None):
        
    
    
    net.eval() # eval mode
    
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [ ]:
print(sample(net, 1000, prime='Anna', top_k=5))

AnnaMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMMM